# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [16]:
import redis

In [17]:
r = redis.Redis(host='localhost', port=6379)

In [18]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [19]:
r.set('greetings', 'hello world')

True

In [20]:
value = r.get("greetings")

In [21]:
print(value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [22]:
r.set("connections",0)

True

In [23]:
value = r.incr("connections")
print(value)

1


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [24]:
r.delete("colors")
r.rpush("colors","red","black","blue")

3

**Q** : Display the second element of the list with `LINDEX`

In [25]:
r.lindex("colors",1)

b'black'

**Q** : Display the full list with `LRANGE`

In [26]:
r.lrange("colors",0,r.llen("colors"))

[b'red', b'black', b'blue']

### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [27]:
r.sadd("nosql","redis","mongodb","orientdb")

3

In [28]:
r.keys()

[b'greetings', b'colors', b'nosql', b'foo', b'connections']

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [31]:
r.sismember("nosql","mysql")

0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [32]:
top14 = {
            "Agen": 10, 
            "Bordeaux":33,
            "Brive":32,
            "Castres":29,
            "Clermont":38,
            "Grenoble":24,
            "La Rochelle":26,
            "Montpellier":32,
            "Oyonnax":14,
            "Pau":20,
            "Racing":40,
            "Stade Français":22,
            "Toulon":36,
            "Toulouse":36
         }

In [33]:
for key,value in top14.items():
    r.zadd("top14",{key:value})

In [34]:
r.zrange("top14",0,r.zcard("top14"),withscores=True)[r.zcard("top14")-1][1]

40.0

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [35]:
r.zscore("top14","Toulon")

36.0

In [36]:
r.zrank("top14","Toulon")

10

**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [37]:
r.zrevrank("top14","Toulon")

3

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [49]:
r.zrange("top14",0,r.zcard("top14"),withscores=True)[::-1][:3]

[(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]

In [50]:
max=r.zrange("top14",0,r.zcard("top14"),withscores=True)[r.zcard("top14")-1][1]
r.zrangebyscore("top14",min=35,max=max)

[b'Toulon', b'Toulouse', b'Clermont', b'Racing']

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [51]:
r.hmset("user:1",{"id":1,"name":"Jean","age":22})

c:\Users\leogo\miniconda3\envs\nosql\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  """Entry point for launching an IPython kernel.


True

In [58]:
r.hgetall("user:1")

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [53]:
r.hset("user:1",'city','Lyon')

1

In [57]:
r.hset("user:1",'name','Paul')

0

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [69]:
r.hset("PUT:my-api.fr/user/10", mapping={'id':10,'name':'jean'})

2

In [70]:
r.hgetall("PUT:my-api.fr/user/10")

{b'id': b'10', b'name': b'jean'}

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [71]:
r.hset("GET:my-api.fr/user?size=2", mapping={"GET":"[{'id':10,'name':'jean'}{'id': 11, 'name': 'Claire'}]"})

1

In [72]:
r.hgetall("GET:my-api.fr/user?size=2")

{b'GET': b"[{'id':10,'name':'jean'}{'id': 11, 'name': 'Claire'}]"}

**Q** : Create a set of cache request keys.

In [73]:
[item for item in r.keys() if "my-api.fr" in str(item)] 

[b'GET:my-api.fr/user?size=2', b'PUT:my-api.fr/user/10']

**Q** : Check if the GET request on http: //http://my-api.fr/user

In [74]:
r.hgetall("GET:my-api.fr/user")

{}

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [75]:
r.delete("PUT:my-api.fr/user/10")

1

In [76]:
[item for item in r.keys() if "my-api.fr" in str(item)] 

[b'GET:my-api.fr/user?size=2']

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [77]:
r.flushall()

True